# MMMI Oversight Script


**TODO**: Get the agents buying/selling.

**TODO**: What are the macro population statistics, given our concept of macro agents including institutional investors, etc.

**TODO**: Reduce NL logging/warning in the notebook.

In [1]:
import HARK.ConsumptionSaving.ConsPortfolioModel as cpm
from HARK.Calibration.Income.IncomeTools import (
     sabelhaus_song_var_profile,
)

import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
import sys

sys.path.append('.')

import hark_portfolio_agents as hpa

In [3]:
sys.path.append('../PNL/py')

import util as UTIL
import pnl as pnl

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Initialize the Simulation

### Initialize market information

Some initial values for the simulation.

$RAP_0$ is the starting risky asset price. This is adjust after every trading day.

In [4]:
risky_asset_price = 100

HARK uses numerical values for wealth. This variable converts "HARK money" to dollars.

In [5]:
dollars_per_hark_money_unit = 1500

Expected rate of return and standard deviation of the market price process will be updated over time.

These are the starting values, derived from the S&P 500.

$\bar{r}_0 = 0.000628$

$sr_0 = 0.011988$

In [6]:
sp500_ror = 0.000628
sp500_std = 0.011988

market_rate_of_return = sp500_ror
market_standard_deviation = sp500_std

Calendar variables. Will ultimately set to be realistic; for now set low for testing.

In [7]:
days_per_quarter = 60
quarters_per_simulation = 6

An attention rate: the chance on any day that an agent will pay attention to the market and trade.

Later, this can be a more realistic function of agent circumstances.

In [8]:
attention_rate = 1 / days_per_quarter

We will need to convert daily price statistics to quarterly price statistics. HARK agents use quarterly (?) statistics. [**Question**: Should HARK agents rebalance knowing how many days are left in the quarter? Or based on a mean number of days between rebalancing periods (based on the attention rate)?]

In [9]:
import math

def ror_quarterly(ror, n_q):
    return pow(1 + ror, n_q) - 1

def sig_quarterly(std, n_q):
    """
     The second formula only holds for special cases (see the attached paper by Andrew Lo), 
     but since we are generating the data we meet this special case.
    """
    return math.sqrt(n_q) * std

### Initialize agents to start

This is to set up the agent parameters and quantities used in the simulation.

`CRRA` refers to the `\rho` parameter in the constant relative risk aversion utility function used by the agents. Utility is over units of resources consumed.

`aNrmInitStd` is normalized initial market assets, standard deviation. The agents' starting wealth is drawn from a lognormal distribution with this standard deviation.

`pLvlInitMean` and `pLvlInitStd` are the mean and standard deviation of the initial level of permanent income of the agents.

In [10]:
## List of tuples: 
##  ( parameters, (i.e. coefficient of relative risk aversion CRRA)
##    number of agents represented,
##    ( initial risky percent, $$$ in risky asset, $$$ in riskless asset)
agent_classes = [
    {
        'CRRA': 2,
        'AgentCount' : 10,
        'aNrmInitMean' : 2
    }] * 10 + \
    [{
        'CRRA': 3, 
        'AgentCount' : 10,
        'aNrmInitMean': 2
    }] * 10 + \
    [{
        'CRRA': 5, 
        'AgentCount' : 10,
        'aNrmInitMean' : 2
    }] * 10 + \
    [{
        'CRRA': 10, 
        'AgentCount' : 10,
        'aNrmInitMean' : 2
    }] * 10 + \
    [{
        'CRRA': 2, 
        'AgentCount' : 10,
        'aNrmInitMean' : 2
    }] * 10


### parameters shared by all agents
agent_parameters = {
    'aNrmInitStd' : 0.0,
    'LivPrb' : [0.98 ** 0.25],
    'PermGroFac': [1.01 ** 0.25],
    'pLvlInitMean' : 1.0, # initial distribution of permanent income
    'pLvlInitStd' : 0.0,
    'Rfree' : 1.0,
    'RiskyAvg': market_rate_of_return,
    'RiskyStd': market_standard_deviation,    
}

Realistic income shock standard deviations from Sabelhaus and Song:

In [11]:
ssvp = sabelhaus_song_var_profile()

# Assume all the agents are 40 for now.
# We will need to make more coherent assumptions about the timing and age of the population later.
# Scaling from annual to quarterly
idx_40 = ssvp['Age'].index(40)

agent_parameters['TranShkStd'] = [ssvp['TranShkStd'][idx_40] / 2]  # Adjust non-multiplicative shock to quarterly
agent_parameters['PermShkStd'] = [ssvp['PermShkStd'][idx_40] ** 0.25]

In [12]:
len(agent_classes)

50

Initialize the agents.

In [13]:
# This is handling this distribution of discount factors based on cstw_MPC in a hacky way
#      This should be corrected with some HARK updates.
#
#
agents = hpa.create_agents(agent_classes, agent_parameters)

In [14]:
agents[0].state_now

{'pLvl': array([2.83275815, 2.83275815]),
 'PlvlAgg': 1.0,
 'bNrm': array([7.09045244, 7.09045244]),
 'mNrm': array([8.09045244, 8.09045244]),
 'aNrm': array([3.23350793, 3.23350793]),
 'aLvl': array([9.15974594, 9.15974594])}

## Get agent risky allocations

Use this stock price to determine the initial number of shares of the risky asset held by each
agent:
 - $RAS_{i0} = (HEC_{i0}.RAW)/ RAP_0$
 
 where RAW is Risky Asset Wealth, amount of resources allocated to the risky asset.
 
 RAS is Risky Asset Shares, number of shares the agent has.

In [15]:
def compute_share_demand(agent):
    agent.solve()
    market_resources = agent.state_now['mNrm']
    permanent_income = agent.state_now['pLvl']
    
    #print("Market Resources: " + str(market_resources))

    # ShareFunc takes normalized market resources as argument
    risky_share = agent.solution[0].ShareFuncAdj(
        market_resources
    )
    
    # denormalize the risky share. See https://github.com/econ-ark/HARK/issues/986
    risky_asset_wealth = risky_share \
                        * market_resources \
                        * permanent_income \
                        * dollars_per_hark_money_unit 
    
    #print("Risky Share: " + str(risky_share))
    #print("Risky Asset Wealth: " + str(risky_asset_wealth))
    
    shares = risky_asset_wealth / risky_asset_price
    
    #print("Shares: " + str(shares))
    
    return shares

In [16]:
for agent in agents:
    agent.shares = compute_share_demand(agent)

## Prepare the main loop

Functions to automate aspects of the main loop.
(Will move this to module code once finalized).

### Data stores

For prices, rates or return, and standard deviations over time.


In [17]:
prices = []
ror_list = []
std_list = []

expected_ror_list = []
expected_std_list = []

buy_sell_list = []

owned_shares_list = []
total_asset_levels_list = []

### Activating agents

"At the beginning of each trading day $t + 1$, select a random group of HARK agents with replacement."

**TODO**: HARK AgentType classes represent multiple agents (as many as `AgentCount`). It is difficult to have the agents within the same AgentType act 'separately', though these agents will have _ex ante_ and _ex post_ heterogeneity of some variables. For the purpose of MMMI, we may need to scaffold the agent model to make it easier to have multiple independently acting agents.

In [18]:
import random

def activate(agents, attention_rate):
    for agent in agents:
        agent.attentive = random.random() < attention_rate

In [19]:
activate(agents, attention_rate)

"**First**, determine the expected return and standard deviation of the risky asset to be used for decision period $t + 1$."

A number of constants used in the calculation of risky expecations.

In [20]:
import math

p1 = 0.1
delta_t1 = 30

a = - math.log(p1) / delta_t1

p2 = 0.1
delta_t2 = 60

b = math.log(p2) / delta_t2

print(a)

print(b)

0.07675283643313485
-0.038376418216567426


In [21]:
def calculate_risky_expectations():
    # note use of data store lists for time tracking here -- not ideal
    D_t = sum([math.exp(a * (l + 1)) for l in range(len(ror_list))])
    S_t = math.exp(b * (len(prices) - 1)) # because p_0 is included in this list.
    
    w_0 = S_t
    w_t = [(1 - S_t) * math.exp(a * (t+1)) / D_t for t in range(len(ror_list))]
    
    print(f"D_t: {D_t}\nS_t / w_0  : {S_t}\nror_list: {ror_list}\n w_t: {w_t}")
    
    expected_ror = w_0 * sp500_ror + sum(
        [w_ror[0] * w_ror[1]
         for w_ror
         in zip(w_t, ror_list)])
    expected_ror_list.append(expected_ror)
    
    expected_std = math.sqrt(
        w_0 * pow(sp500_std, 2) \
        +  sum([w_ror_er[0] * pow(w_ror_er[1] - expected_ror, 2)
                for w_ror_er
                in zip(w_t, ror_list)]))
    expected_std_list.append(expected_std)
    
    print(f'daily expectations: {expected_ror},{expected_std}')

    # TODO: This should be an adaptive function of market prices
    # as described in John's Experiment design.
    # This is a stopgap for now.
    market_risky_params = {
        'RiskyAvg': ror_quarterly(expected_ror, days_per_quarter),
        'RiskyStd': sig_quarterly(expected_std, days_per_quarter)
    }
    
    return market_risky_params


"**Second**, using [these expectations] calculate the total number of shares of the risky asset that the selected agents want to buy and sell."

In [22]:
def compute_total_buy_sell(agents):
    """
    TODO: Should this be done without making so many changes to the underlying
    agent? Could be done using copies.
    """
    risky_expectations = calculate_risky_expectations()
    
    print(risky_expectations)
    
    buy_shares_total = 0
    sell_shares_total = 0
    
    for agent in agents:
        if agent.attentive:
            # Note: this mutates the underlying agent
            agent.assign_parameters(**risky_expectations)
            
            d_shares = compute_share_demand(agent)
            
            delta_shares = d_shares - agent.shares
            
            # NOTE: This mutates the agent
            agent.shares = d_shares
            
            buy_shares_total += delta_shares[delta_shares > 0].sum()
            sell_shares_total += -delta_shares[delta_shares < 0].sum()
            
    return (round(buy_shares_total), round(sell_shares_total))

In [23]:
compute_total_buy_sell(agents)

D_t: 0
S_t / w_0  : 1.0391223038351691
ror_list: []
 w_t: []
daily expectations: 0.0006525688068084862,0.012220249349435207
{'RiskyAvg': 0.03991747452054173, 'RiskyStd': 0.09465764443372916}


(0, 0)

"**Third**, using [total buy and sell shares] as input to the Buy Broker and Sell Broker, run a day of NetLogo trading and determine the end of day price of the risky asset."

In [24]:
import os

def run_market(config, buy_sell, seed = None):    
    pnl.run_NLsims(
        config,
        broker_buy_limit = buy_sell[0],
        broker_sell_limit = buy_sell[1],
        SEED = seed,
        use_cache = True
    )

"**Fourth**, use the final price to calculate the amount of wealth each (all) HARK agent has allocated to the risky asset."

In [25]:
def get_transactions(config, seed = 0, buy_sell = (0,0)):
    logfile = pnl.transaction_file_name(config, seed, buy_sell[0], buy_sell[1])
    
    # use run_market() first to create logs
    transactions = pd.read_csv(
        logfile,
        delimiter='\t'
    )
    return transactions

def get_last_simulation_price(config, seed = 0, buy_sell = (0,0)):
    """
    Dividing the output of this by 4
    """
    
    transactions = get_transactions(config, seed=seed, buy_sell = buy_sell)
    prices = transactions['TrdPrice']
    
    return prices[prices.index.values[-1]]

In [26]:
netlogo_ror = -0.00052125
netlogo_std =  0.0068
simulation_price_scale = 0.25

def daily_rate_of_return(config = None, seed = 0, buy_sell = (0, 0)):
    
    last_sim_price = get_last_simulation_price(config, seed=seed, buy_sell = buy_sell)
    
    ror = (last_sim_price * simulation_price_scale - 100) / 100
    
    # adjust to calibrated NetLogo to S&P500
    ror = sp500_std * (ror - netlogo_ror) / netlogo_std + sp500_ror
    
    return ror

In [27]:
def update_agent_wealth(old_share_price, ror):

    new_share_price = old_share_price * (1 + ror)

    for agent in agents:
        market_resources = agent.state_now['mNrm'] * agent.state_now['pLvl']
    
        old_raw = agent.shares * old_share_price
        new_raw = agent.shares * new_share_price
    
        delta_mNrm = (new_raw - old_raw) / \
            (dollars_per_hark_money_unit * agent.state_now['pLvl'])
    
        agent.state_now['mNrm'] = agent.state_now['mNrm'] + delta_mNrm

"Repeat each trading day until the end of the quarter. Then update the economic conditions for each HARK agent (e.g., permanent income level), and repeat [daily loop]."

## Run the main loop



In [28]:
import itertools

seeds = itertools.cycle([0,1,2,3,4])

config = UTIL.read_config(
    config_file = "../PNL/macroliquidity.ini",
    config_local_file = "../PNL/macroliquidity_local.ini"
)

total_asset_levels = sum([agent.state_now['aLvl'].sum() for agent in agents])
print("Total Asset Levels: " + str(total_asset_levels))
total_asset_levels_list.append(total_asset_levels)

# Main loop
for quarter in range(quarters_per_simulation):
    print(f"Q-{quarter}")

    agent_id = 0
    
    for agent in agents:
        agent_id = agent_id + 1
        agent.solve()
        agent.simulate(sim_periods=1)
    
    for day in range(days_per_quarter):
        print(f"Q-{quarter}:D-{day}")
        
        # Set to a number for a fixed seed, or None to rotate
        seed = next(seeds)

        prices.append(risky_asset_price)
        
        print(f"Price history ({len(prices)}): {prices}")
        activate(agents, attention_rate)
        buy_sell = compute_total_buy_sell(agents)
        print("Buy/Sell Limit: " + str(buy_sell))
        buy_sell_list.append(buy_sell)
        
        run_market(config, buy_sell, seed = seed)
        
        ror =  daily_rate_of_return(config = config, seed = seed, buy_sell = buy_sell)
        print("ror: " + str(ror))
        ror_list.append(ror)
        
        update_agent_wealth(risky_asset_price, ror)
        
        owned_shares = sum([sum(agent.shares) for agent in agents])
        print("Owned: " + str(owned_shares))
        owned_shares_list.append(owned_shares)
        
        total_asset_levels = sum([agent.state_now['aLvl'].sum() for agent in agents])
        print("Total Asset Levels: " + str(total_asset_levels))
        total_asset_levels_list.append(total_asset_levels)
        
        risky_asset_price = risky_asset_price * (1 + ror)
        
        print(f"Price day {day}: {risky_asset_price}")

config_file: ../PNL/macroliquidity.ini
config_local_file: ../PNL/macroliquidity_local.ini
Total Asset Levels: 4598.908576099033
Q-0
Q-0:D-0
Price history (1): [100]
D_t: 0
S_t / w_0  : 1.0
ror_list: []
 w_t: []
daily expectations: 0.000628,0.011988
{'RiskyAvg': 0.03838661143795652, 'RiskyStd': 0.09285864870866903}
Buy/Sell Limit: (0, 0)
Output for S:0,BL:0,SL:0 already exists.
Will use cache.
ror: 0.005954286029411765
Owned: 0.0
Total Asset Levels: 2358.6697165635105
Price day 0: 100.59542860294117
Q-0:D-1
Price history (2): [100, 100.59542860294117]
D_t: 1.0797751623277096
S_t / w_0  : 0.9623506263980885
ror_list: [0.005954286029411765]
 w_t: [0.03764937360191145]
daily expectations: 0.0008285313326319651,0.011802145880789828
{'RiskyAvg': 0.05094661634272635, 'RiskyStd': 0.09141902889161894}
Buy/Sell Limit: (0, 0)
Output for S:1,BL:0,SL:0 already exists.
Will use cache.
ror: 0.0015469330882352942
Owned: 0.0
Total Asset Levels: 2358.6697165635105
Price day 1: 100.75104299997228
Q-0:D-2

Buy/Sell Limit: (0, 0)
Output for S:1,BL:0,SL:0 already exists.
Will use cache.
ror: 0.0015469330882352942
Owned: 0.0
Total Asset Levels: 2358.6697165635105
Price day 11: 105.05198053185556
Q-0:D-12
Price history (13): [100, 100.59542860294117, 100.75104299997228, 101.35094352775567, 101.50772665582264, 102.11213269452675, 102.72013753966321, 102.87903871925141, 103.49160994221675, 103.65170453799112, 104.2688764342464, 104.88972314850129, 105.05198053185556]
D_t: 20.463730431511458
S_t / w_0  : 0.6309573444801932
ror_list: [0.005954286029411765, 0.0015469330882352942, 0.005954286029411765, 0.0015469330882352942, 0.005954286029411765, 0.005954286029411765, 0.0015469330882352942, 0.005954286029411765, 0.0015469330882352942, 0.005954286029411765, 0.005954286029411765, 0.0015469330882352942]
 w_t: [0.019472651606871085, 0.021026085549760164, 0.022703444937608595, 0.024514615942904532, 0.0264702734091512, 0.028581943767225098, 0.030862072970896947, 0.03332409985191988, 0.03598253532703159,

Buy/Sell Limit: (0, 0)
Output for S:3,BL:0,SL:0 already exists.
Will use cache.
ror: 0.0015469330882352942
Owned: 0.0
Total Asset Levels: 2358.6697165635105
Price day 18: 108.07646772670813
Q-0:D-19
Price history (20): [100, 100.59542860294117, 100.75104299997228, 101.35094352775567, 101.50772665582264, 102.11213269452675, 102.72013753966321, 102.87903871925141, 103.49160994221675, 103.65170453799112, 104.2688764342464, 104.88972314850129, 105.05198053185556, 105.67749007189842, 105.84096607797231, 106.47117346362982, 107.1051332843194, 107.27081775891676, 107.90953889046226, 108.07646772670813]
D_t: 44.64815322660753
S_t / w_0  : 0.4823178482239307
ror_list: [0.005954286029411765, 0.0015469330882352942, 0.005954286029411765, 0.0015469330882352942, 0.005954286029411765, 0.005954286029411765, 0.0015469330882352942, 0.005954286029411765, 0.0015469330882352942, 0.005954286029411765, 0.005954286029411765, 0.0015469330882352942, 0.005954286029411765, 0.0015469330882352942, 0.005954286029411

Buy/Sell Limit: (0, 0)
Output for S:3,BL:0,SL:0 already exists.
Will use cache.
ror: 0.0015469330882352942
Owned: 0.0
Total Asset Levels: 2358.6697165635105
Price day 23: 110.35918613665358
Q-0:D-24
Price history (25): [100, 100.59542860294117, 100.75104299997228, 101.35094352775567, 101.50772665582264, 102.11213269452675, 102.72013753966321, 102.87903871925141, 103.49160994221675, 103.65170453799112, 104.2688764342464, 104.88972314850129, 105.05198053185556, 105.67749007189842, 105.84096607797231, 106.47117346362982, 107.1051332843194, 107.27081775891676, 107.90953889046226, 108.07646772670813, 108.71998592860143, 109.36733582193395, 109.53651977248904, 110.18873154188076, 110.35918613665358]
D_t: 71.86629724049477
S_t / w_0  : 0.39810717055349726
ror_list: [0.005954286029411765, 0.0015469330882352942, 0.005954286029411765, 0.0015469330882352942, 0.005954286029411765, 0.005954286029411765, 0.0015469330882352942, 0.005954286029411765, 0.0015469330882352942, 0.005954286029411765, 0.0059

Buy/Sell Limit: (0, 0)
Output for S:2,BL:0,SL:0 already exists.
Will use cache.
ror: 0.005954286029411765
Owned: 0.0
Total Asset Levels: 2358.6697165635105
Price day 27: 112.51606376646113
Q-0:D-28
Price history (29): [100, 100.59542860294117, 100.75104299997228, 101.35094352775567, 101.50772665582264, 102.11213269452675, 102.72013753966321, 102.87903871925141, 103.49160994221675, 103.65170453799112, 104.2688764342464, 104.88972314850129, 105.05198053185556, 105.67749007189842, 105.84096607797231, 106.47117346362982, 107.1051332843194, 107.27081775891676, 107.90953889046226, 108.07646772670813, 108.71998592860143, 109.36733582193395, 109.53651977248904, 110.18873154188076, 110.35918613665358, 111.0162962968843, 111.67731907896187, 111.85007641905054, 112.51606376646113]
D_t: 102.55588180882256
S_t / w_0  : 0.3414548873833602
ror_list: [0.005954286029411765, 0.0015469330882352942, 0.005954286029411765, 0.0015469330882352942, 0.005954286029411765, 0.005954286029411765, 0.0015469330882352

...

Buy/Sell Limit: (0, 0)
Output for S:0,BL:0,SL:0 already exists.
Will use cache.
ror: 0.005954286029411765
Owned: 0.0
Total Asset Levels: 2358.6697165635105
Price day 50: 123.98002344511723
Q-0:D-51
Price history (52): [100, 100.59542860294117, 100.75104299997228, 101.35094352775567, 101.50772665582264, 102.11213269452675, 102.72013753966321, 102.87903871925141, 103.49160994221675, 103.65170453799112, 104.2688764342464, 104.88972314850129, 105.05198053185556, 105.67749007189842, 105.84096607797231, 106.47117346362982, 107.1051332843194, 107.27081775891676, 107.90953889046226, 108.07646772670813, 108.71998592860143, 109.36733582193395, 109.53651977248904, 110.18873154188076, 110.35918613665358, 111.0162962968843, 111.67731907896187, 111.85007641905054, 112.51606376646113, 112.69011858845947, 113.36110778722349, 114.03609224759958, 114.21249845195045, 114.89255233586711, 115.07028342666727, 115.75544480767513, 116.44468583552181, 116.62481797298996, 117.31923549732922, 117.50072050460652,

Buy/Sell Limit: (0, 0)
Output for S:3,BL:0,SL:0 already exists.
Will use cache.
ror: 0.0015469330882352942
Owned: 0.0
Total Asset Levels: 2358.6697165635105
Price day 53: 125.10439594947447
Q-0:D-54
Price history (55): [100, 100.59542860294117, 100.75104299997228, 101.35094352775567, 101.50772665582264, 102.11213269452675, 102.72013753966321, 102.87903871925141, 103.49160994221675, 103.65170453799112, 104.2688764342464, 104.88972314850129, 105.05198053185556, 105.67749007189842, 105.84096607797231, 106.47117346362982, 107.1051332843194, 107.27081775891676, 107.90953889046226, 108.07646772670813, 108.71998592860143, 109.36733582193395, 109.53651977248904, 110.18873154188076, 110.35918613665358, 111.0162962968843, 111.67731907896187, 111.85007641905054, 112.51606376646113, 112.69011858845947, 113.36110778722349, 114.03609224759958, 114.21249845195045, 114.89255233586711, 115.07028342666727, 115.75544480767513, 116.44468583552181, 116.62481797298996, 117.31923549732922, 117.50072050460652

Buy/Sell Limit: (0, 0)
Output for S:1,BL:0,SL:0 already exists.
Will use cache.
ror: 0.0015469330882352942
Owned: 0.0
Total Asset Levels: 2358.6697165635105
Price day 56: 126.79448568949174
Q-0:D-57
Price history (58): [100, 100.59542860294117, 100.75104299997228, 101.35094352775567, 101.50772665582264, 102.11213269452675, 102.72013753966321, 102.87903871925141, 103.49160994221675, 103.65170453799112, 104.2688764342464, 104.88972314850129, 105.05198053185556, 105.67749007189842, 105.84096607797231, 106.47117346362982, 107.1051332843194, 107.27081775891676, 107.90953889046226, 108.07646772670813, 108.71998592860143, 109.36733582193395, 109.53651977248904, 110.18873154188076, 110.35918613665358, 111.0162962968843, 111.67731907896187, 111.85007641905054, 112.51606376646113, 112.69011858845947, 113.36110778722349, 114.03609224759958, 114.21249845195045, 114.89255233586711, 115.07028342666727, 115.75544480767513, 116.44468583552181, 116.62481797298996, 117.31923549732922, 117.50072050460652

Buy/Sell Limit: (0, 0)
Output for S:3,BL:0,SL:0 already exists.
Will use cache.
ror: 0.0015469330882352942
Owned: 0.0
Total Asset Levels: 2358.6697165635105
Price day 58: 127.74676679861352
Q-0:D-59
Price history (60): [100, 100.59542860294117, 100.75104299997228, 101.35094352775567, 101.50772665582264, 102.11213269452675, 102.72013753966321, 102.87903871925141, 103.49160994221675, 103.65170453799112, 104.2688764342464, 104.88972314850129, 105.05198053185556, 105.67749007189842, 105.84096607797231, 106.47117346362982, 107.1051332843194, 107.27081775891676, 107.90953889046226, 108.07646772670813, 108.71998592860143, 109.36733582193395, 109.53651977248904, 110.18873154188076, 110.35918613665358, 111.0162962968843, 111.67731907896187, 111.85007641905054, 112.51606376646113, 112.69011858845947, 113.36110778722349, 114.03609224759958, 114.21249845195045, 114.89255233586711, 115.07028342666727, 115.75544480767513, 116.44468583552181, 116.62481797298996, 117.31923549732922, 117.50072050460652

KeyboardInterrupt: 

In [ ]:
len(ror_list)

In [ ]:
data = pd.DataFrame.from_dict({
    't' : range(1 + len(prices)),
    'prices' : [100] + prices,
    'buy' : [None] + [bs[0] for bs in buy_sell_list],
    'sell' : [None]  + [bs[1] for bs in buy_sell_list],
    'owned' : [None] + owned_shares_list,
    'total_assets' : total_asset_levels_list,
    'ror' : [None] + ror_list,
    'expected_ror' : expected_ror_list,
    #'expected_ror_q' : [ror_quarterly(er, days_per_quarter) for er in expected_ror_list],
    'expected_std' : expected_std_list,
    #'expected_std_q' : [sig_quarterly(es, days_per_quarter) for es in expected_std_list],
})


data

In [ ]:
data.corr()

In [ ]:
import seaborn as sns

sns.pairplot(data)

In [ ]:
data['sell'].max()

In [ ]:
500 / 348


In [ ]:
sns.lineplot(data= data, x = 't', y='total_assets')

In [ ]:
sns.lineplot(data= data, x = 't', y='expected_ror')

In [ ]:
sns.lineplot(data= data, x = 't', y='expected_std')